<a href="https://colab.research.google.com/github/JunHyeong-data/python-data-analysis-book-study/blob/main/ch10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 집계와 그룹 연산

- 데이터셋을 분류하고 각 그룹별로 집계나 변형 같이 어떤 함수를 적용하는 작업은 데이터 분석 과정에서 중요한 일
- 데이터를 불러오고 취합해서 하나의 데이터셋을 준비한 후 그룹 통계를 구하거나 가능하다면 피벗 테이블을 구해 보고서를 만들고 시각화
- 판다스는 데이터셋을 자연스럽게 나누고 요약하는 groupby라는 유연한 방법을 제공
- 관계형 데이터베이스와 SQL이 인기 있는 이유는 데이터를 쉽게 합치고 걸러내고 변형하고 집계 가능
- 복잡한 그룹 연산도 각 그룹에 연관된 데이터를 조작하는 파이썬 함수를 조합해 해결
- 이 장에서 배우는 내용
  - 하나 이상의 키를 이용해서 판다스 객체를 여러 조각으로 나누는 방법
  - 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
  - 정규화, 선형 회귀, 순위 또는 부분집합 선택 같은 그룹 내 변형이나 다른 조작을 적용하는 방법
  - 피벗 테이블과 교차표를 구하는 방법
  - 사분위수 분석과 다른 데이터 파생 그룹 분석을 수행하는 방법

## 그룹 연산에 대한 고찰
- 해들리 위컴은 그룹 연산에 대해 **분리-적용-결합**이라는 새로운 용어를 만듬
- 판다스 객체나 다른 객체에 들어 있는 데이터를 하나 이상의 키 기준으로 **분리**한다
- 분리하고 나면 함수를 각 그룹에 **적용**시켜 새로운 값을 얻어냄
- 마지막으로 함수를 적용한 결과를 하나의 객체로 **결합**한다

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                      dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,0.770178,0.061994
1,a,2,-1.224717,1.479310
2,None,1,-0.083958,-0.267685
3,b,2,0.761390,-0.960546
4,b,1,1.115919,-1.970858
5,a,<NA>,0.440322,0.830387
6,None,1,-0.253513,-0.718656


✅ groupby란?
groupby는 다음 과정을 세 단계로 나눠서 생각할 수 있어:

1.분할 (Split): 데이터를 어떤 기준으로 그룹 나눔 (df["key1"])

2.적용 (Apply): 각 그룹에 대해 연산 수행 (sum(), mean(), count() 등)

3.결합 (Combine): 결과를 하나로 합침

이 개념을 "Split → Apply → Combine" 이라고 불러.

In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()
# key1을 기준으로 그룹을 나눔

,data1
key1,
a,-0.004739
b,0.938654


In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means
# key1과 key2라는 두 개의 컬럼을 기준으로 data1을 그룹화하고,
# 각 그룹의 평균을 계산해서 means에 저장하는 거야.

key1  key2
a     1       0.770178
      2      -1.224717
b     1       1.115919
      2       0.761390
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,1,2
key1,,
a,0.770178,-1.224717
b,1.115919,0.761390


In [7]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

CA  2005   -0.392197
    2006   -0.083958
OH  2005    0.765784
    2006    0.431203
Name: data1, dtype: float64

In [8]:
df.groupby("key1").mean()
df.groupby("key2").mean(numeric_only=True)
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1     0.770178  0.061994
     2    -1.224717  1.479310
b    1     1.115919 -1.970858
     2     0.761390 -0.960546

In [9]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [10]:
print(df.groupby("key1", dropna=False).size())
df.groupby(["key1", "key2"], dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64


key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [11]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### 그룹 간 순회하기

In [12]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.770178  0.061994
1    a     2 -1.224717  1.479310
5    a  <NA>  0.440322  0.830387
b
  key1  key2     data1     data2
3    b     2  0.761390 -0.960546
4    b     1  1.115919 -1.970858


In [13]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2     data1     data2
0    a     1  0.770178  0.061994
('a', np.int64(2))
  key1  key2     data1    data2
1    a     2 -1.224717  1.47931
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1  1.115919 -1.970858
('b', np.int64(2))
  key1  key2    data1     data2
3    b     2  0.76139 -0.960546


In [14]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.761390,-0.960546
4,b,1,1.115919,-1.970858


In [15]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")

<ipython-input-15-750e76e37905>:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


In [16]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0  0.770178  0.061994
1 -1.224717  1.479310
2 -0.083958 -0.267685
3  0.761390 -0.960546
4  1.115919 -1.970858
5  0.440322  0.830387
6 -0.253513 -0.718656
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


### 열이나 열의 일부만 선택하기

In [17]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.061994
     2     1.479310
b    1    -1.970858
     2    -0.960546

In [18]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped
s_grouped.mean()

key1  key2
a     1       0.061994
      2       1.479310
b     1      -1.970858
      2      -0.960546
Name: data2, dtype: float64

### 딕셔너리와 Series에서 그룹화하기

In [19]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.163938,-0.050177,0.748805,-1.894970,-0.196564
Steve,-0.115173,1.281037,0.057624,2.630070,1.571500
Wanda,1.102795,NaN,NaN,0.472185,-0.360299
Jill,1.030659,0.123412,0.942851,0.302428,-0.875857
Trey,-0.080026,0.002650,-0.550555,0.570519,-0.578077


In [20]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [21]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

<ipython-input-21-acba1b50385d>:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,-1.146164,-0.410678
Steve,2.687694,2.737365
Wanda,0.472185,0.742496
Jill,1.245279,0.278214
Trey,0.019963,-0.655453


In [22]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis="columns").count()

<ipython-input-22-546a561aa46f>:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### 함수로 그룹화하기

In [23]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.163938,-0.050177,0.748805,-1.894970,-0.196564
4,0.950633,0.126062,0.392295,0.872947,-1.453933
5,0.987622,1.281037,0.057624,3.102255,1.211201


In [24]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-0.163938,-0.050177,0.748805,-1.894970,-0.196564
4,two,-0.080026,0.002650,-0.550555,0.302428,-0.875857
5,one,-0.115173,1.281037,0.057624,0.472185,-0.360299


### 색인 단계로 그룹화하기

In [25]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      3.200543 -1.129004 -2.773242  1.716713  1.647826
1      1.063106  0.418717  0.130946 -0.930833 -0.774025
2      0.692684  1.037449 -0.180537  0.253011  0.427688
3      0.948782  0.895526 -2.575798 -0.752203  0.816635

In [26]:
hier_df.groupby(level="cty", axis="columns").count()

<ipython-input-26-a23a9a6fd77b>:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 데이터 집계
- 데이터 집계는 배열로부터 스칼라 값을 만들어내는 모든 데이터 변환 작업을 말한다

In [27]:
df
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     1   -1.224717
      5    0.440322
b     3    0.761390
      4    1.115919
Name: data1, dtype: float64

In [28]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.994895,1.417316
b,1,0.354529,1.010312


In [29]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -0.004739  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.938654  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.605250  0.770178   3.0  0.790564  0.709497  0.061994  0.446191   
b     1.027287  1.115919   2.0 -1.465702  0.714399 -1.970858 -1.718280   

                                    
           50%       75%       max  
key1                                
a     0.830387  1.154849  1.479310  
b    -1.465702 -1.213124 -0.960546  

[2 rows x 24 columns]

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
!rm examples
!ln -s "/content/drive/My Drive/Colab Notebooks/pydata-book-3rd-edition/examples" examples

In [39]:
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [40]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [41]:
grouped = tips.groupby(["day", "smoker"])

In [42]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [43]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [44]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

<ipython-input-44-006287cd2d77>:1: FutureWarning: The provided callable <function std at 0x7a5219bfdee0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([("average", "mean"), ("stdev", np.std)])


average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [45]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [46]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [47]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

<ipython-input-47-c345fa286676>:2: FutureWarning: The provided callable <function var at 0x7a5219bfe020> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [48]:
grouped.agg({"tip" : np.max, "size" : "sum"})
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

<ipython-input-48-bb1125719dff>:1: FutureWarning: The provided callable <function max at 0x7a5219bfd3a0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip" : np.max, "size" : "sum"})


tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 색인되지 않은 형태로 집계된 데이터 반환하기

In [49]:
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## apply 메서드: 일반적인 분리-적용-병합

In [50]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [51]:
tips.groupby("smoker").apply(top)

<ipython-input-51-2a8c000674ed>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [52]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

<ipython-input-52-b9b37c813d89>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [53]:
result = tips.groupby("smoker")["tip_pct"].describe()
result
result.unstack("smoker")

smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### 그룹 키 생략하기

In [54]:
tips.groupby("smoker", group_keys=False).apply(top)

<ipython-input-54-de6df242f7c0>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)


,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### 사분위수 분석과 버킷 분석

In [55]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

,data1
0,"(-0.237, 1.435]"
1,"(-0.237, 1.435]"
2,"(1.435, 3.107]"
3,"(-0.237, 1.435]"
4,"(-0.237, 1.435]"
5,"(-1.909, -0.237]"
6,"(-0.237, 1.435]"
7,"(-1.909, -0.237]"
8,"(-1.909, -0.237]"
9,"(-1.909, -0.237]"


In [56]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

<ipython-input-56-da7b0ef9744c>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


min       max  count      mean
data1                                                      
(-3.587, -1.909] data1 -3.580639 -1.927864     23 -2.341111
                 data2 -2.075660  2.305313     23 -0.111715
(-1.909, -0.237] data1 -1.904816 -0.237867    376 -0.859075
                 data2 -2.603576  3.092488    376  0.048155
(-0.237, 1.435]  data1 -0.229848  1.422950    520  0.455483
                 data2 -3.079348  3.226658    520 -0.053637
(1.435, 3.107]   data1  1.440481  3.107305     81  1.852644
                 data2 -2.696098  2.490022     81  0.133463

In [57]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.587, -1.909] -3.580639 -1.927864    23 -2.341111 -2.075660  2.305313   
(-1.909, -0.237] -1.904816 -0.237867   376 -0.859075 -2.603576  3.092488   
(-0.237, 1.435]  -0.229848  1.422950   520  0.455483 -3.079348  3.226658   
(1.435, 3.107]    1.440481  3.107305    81  1.852644 -2.696098  2.490022   

                                  
                 count      mean  
data1                             
(-3.587, -1.909]    23 -0.111715  
(-1.909, -0.237]   376  0.048155  
(-0.237, 1.435]    520 -0.053637  
(1.435, 3.107]      81  0.133463

In [58]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.580639 -0.649500    250 -1.252192
      data2 -2.512979  3.092488    250  0.057573
1     data1 -0.642919  0.014030    250 -0.297946
      data2 -2.846010  2.519407    250 -0.088527
2     data1  0.016475  0.659441    250  0.314179
      data2 -3.027118  3.027597    250 -0.012145
3     data1  0.660833  3.107305    250  1.276188
      data2 -3.079348  3.226658    250  0.036923

### 그룹별 값으로 결측치 채우기

In [59]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s
s.fillna(s.mean())

,0
0,0.221663
1,-0.453715
2,0.221663
3,0.736832
4,0.221663
5,0.381870


In [60]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

,0
Ohio,-0.240425
New York,0.740492
Vermont,-1.933911
Florida,0.178526
Oregon,1.625630
Nevada,-0.513147
California,0.909943
Idaho,0.304014


In [61]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data
data.groupby(group_key).size()
data.groupby(group_key).count()
data.groupby(group_key).mean()

,0
East,0.226198
West,1.267786


In [62]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio         -0.240425
      New York      0.740492
      Vermont       0.226198
      Florida       0.178526
West  Oregon        1.625630
      Nevada        1.267786
      California    0.909943
      Idaho         1.267786
dtype: float64

In [63]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

East  Ohio         -0.240425
      New York      0.740492
      Vermont       0.500000
      Florida       0.178526
West  Oregon        1.625630
      Nevada       -1.000000
      California    0.909943
      Idaho        -1.000000
dtype: float64

### 랜덤 표본과 순열

In [64]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [65]:
deck.head(13)

,0
AH,1
2H,2
3H,3
4H,4
5H,5
6H,6
7H,7
8H,8
9H,9
10H,10


In [66]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

,0
10D,10
5S,5
QD,10
KS,10
5D,5


In [67]:
def get_suit(card):
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  AC     1
   2C     2
D  5D     5
   9D     9
H  9H     9
   5H     5
S  3S     3
   JS    10
dtype: int64

In [68]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

,0
8C,8
KC,10
6D,6
5D,5
4H,4
5H,5
QS,10
10S,10


### 그룹 가중평균과 상관관계

In [69]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

,category,data,weights
0,a,-0.367796,0.789412
1,a,-1.024878,0.170810
2,a,-0.026848,0.528158
3,a,0.703046,0.905748
4,b,0.501865,0.261847
5,b,0.540090,0.764967
6,b,-0.521053,0.899597
7,b,-0.173800,0.477137


In [70]:
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

grouped.apply(get_wavg)

<ipython-input-70-d63cd288ca3c>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


,0
category,
a,0.065661
b,-0.002955


In [71]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True,
                       index_col=0)
close_px.info()
close_px.tail(4)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


## 그룹 변환과 래핑되지 않은 groupby

In [72]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [73]:
g = df.groupby('key')['value']
g.mean()

,value
key,
a,4.5
b,5.5
c,6.5


In [74]:
def get_mean(group):
    return group.mean()
g.transform(get_mean)

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [75]:
g.transform('mean')

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [76]:
def times_two(group):
    return group * 2
g.transform(times_two)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


In [77]:
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


In [78]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [79]:
g.transform(normalize)
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [80]:
g.transform('mean')
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


## 피벗 테이블과 교차표

In [81]:
tips.head()
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [82]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [83]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [84]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [85]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [86]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")

In [87]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [88]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [89]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

## 마치며
- 판다스의 데이터 그룹화 도구를 마스터한다면 데이터 정제뿐 아니라 모델링이나 통계분석에 도움
- 다음은 시계열 데이터를 알아보자